# 16 协程

- 生成器作为协程使用时的行为和状态

- 使用装饰器自动预激协程

- 调用方如何使用生成器对象的 .close() 和 .throw(...) 方法 控制协程

- 协程终止时如何返回值

- yield from 新句法的用途和语义

- 使用案例——使用协程管理仿真系统中的并发活动

## 16.1 生成器如何进化成协程

- .send() : 生成器的调用方可以使用.send(...)方法发送数据，发送的数据会成为生成器函数中yield表达式的值。

- .throw() : 让调用方抛出异常，在生成器中处理

- .close() : 后者的作用是终止生成器。

## 16.2 用作协程的生成器的基本行为

In [6]:
def simple_coroutine():
    print("-> coroutine started :")
    x = yield   # 在协程中，yield通常出现在表达式的右边（例如，datum=yield），
                # 可以产出值，也可以不产出——如果yield关键字后面没有表达式，那么生成器产出None。
    print("-> coroutine received :", x)

my_coro = simple_coroutine()
my_coro # <generator object simple_coroutine at 0x06D40C30>

<generator object simple_coroutine at 0x063B98E8>

In [7]:
# 最先调用next(my_coro)函数这一步通常称为“预激”（prime）协程
# （即，让协程向前执行到第一个yield表达式，准备好作为活跃的协程使用）。
next(my_coro) # 调用next，使函数运行到yield关键字处，启动生成器！

-> coroutine started :


In [8]:
my_coro.send(42)  # 将值传递给协程！并执行到下一个yield关键字，直至StopIteration

-> coroutine received : 42


StopIteration: 

### 协程的四个状态
当前状态可以使用inspect.getgeneratorstate(...)函数确定：

- 'GEN_CREATED'   ：等待开始执行
- 'GEN_RUNNING'   ：解释器正在执行
- 'GEN_SUSPENDED' ：在yield表达式处暂停
- 'GEN_CLOSED'    ：执行结束

In [ ]:
# 最先调用next(my_coro)函数这一步通常称为“预激”（prime）协程
# （即，让协程向前执行到第一个yield表达式，准备好作为活跃的协程使用）。
my_coro1 = simple_coroutine()
# 这里需要先调用 next(my_coro1) !
my_coro1.send(45)
# Error:
# TypeError: can't send non-None value to a just-started generator

TypeError: can't send non-None value to a just-started generator

In [ ]:
# 产出多个值的例子
def simple_coro2(a):
    print('-> Started: a = ', a)
    b = yield a
    print('-> Started: b = ', b)
    c = yield a+b
    print('-> Started: c = ', c)

my_coro2 = simple_coro2(14)

In [ ]:
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

# 'GEN_CREATED'

'GEN_CREATED'

In [ ]:
next(my_coro2)

# -> Started: a =  14
# 14

-> Started: a =  14


14

> 值得注意的是，b = yield a 这里并没有将14赋值给b，b的类型依然是None!
>
> 见下面的测试！\# Test
>
> 关键的一点是，协程在yield关键字所在的位置暂停执行。前面说过，在赋值语句中，=右边的代码在赋值之前执行。
> 因此，对于b=yield a这行代码来说，等到客户端代码再激活协程时才会设定b的值。这种行为要花点时间才能习惯，
> 不过一定要理解，这样才能弄懂异步编程中yield的作用（后文探讨）。

```
b = yield a

# 执行顺序：

yield a (a 为创建函数时传入的值 14）
此时调用方使用send(28)
接受该28的变量时yield 等号左边的变量b
即
b = send(28)
故 b = 28
因为send方法的参数会成为暂停的yield表达式的值，所以，仅当协程处于暂停状态时才能调用send方法，

这也是实现异步同步的方式！
```
![coro](./img/coro.png)

In [ ]:
getgeneratorstate(my_coro2)
# 'GEN_SUSPENDED' 在yield表达式处暂停

'GEN_SUSPENDED'

In [ ]:
my_coro2.send(28)
# -> Started: b =  28
# 42

-> Started: b =  28


42

In [ ]:
getgeneratorstate(my_coro2)
# 'GEN_SUSPENDED' 在yield表达式处暂停
# 在第一次执行yield之前，需要next来预激协程，后面使用send既可以自动直行道yield处暂停（即不再需要next）
# 因此，对于b=yield a这行代码来说，等到客户端代码再激活协程时才会设定b的值。这种行为要花点时间才能习惯，

'GEN_SUSPENDED'

In [ ]:
my_coro2.send(99)
# -> Started: c =  99
# StopIteration

In [ ]:
getgeneratorstate(my_coro2)

# 'GEN_CLOSED'

'GEN_CLOSED'

In [ ]:
# Test

my_coro2_2 = simple_coro2(14)
next(my_coro2_2)

-> Started: a =  14


14

In [ ]:
next(my_coro2_2)
# -> Started: b =  None

# 在赋值语句中，=右边的代码在赋值之前执行。
# 因此，对于b=yield a这行代码来说，等到客户端代码再激活协程时才会设定b的值。这种行为要花点时间才能习惯，
# 不过一定要理解，这样才能弄懂异步编程中yield的作用（后文探讨）。

-> Started: b =  None


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

## 16.3 示例：使用协程计算移动平均值

In [ ]:
# 定义一个计算移动平均值的协程

def averager():
    total = 0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

a = averager()
next(a) # 预激协程！
a.send(1)

1.0

In [ ]:
a.send(2)

1.5

In [ ]:
a.send(3)

2.0

在上述doctest中（示例16-4），调用next(coro_avg)函数后，协程会向前执行到yield表达式，
产出average变量的初始值——None，因此不会出现在控制台中。此时，协程在yield表达式处暂停，
等到调用方发送值。coro_avg.send(10)那一行发送一个值，激活协程，把发送的值赋给term，
并更新total、count和average三个变量的值，然后开始while循环的下一次迭代，
产出average变量的值，等待下一次为term变量赋值。

## 16.4 预激协程的装饰器

如果不预激，那么协程没什么用。调用my_coro.send(x)之前，记住一定要调用next(my_coro)。
为了简化协程的用法，有时会使用一个预激装饰器。

In [10]:
# 预激协程装饰器
from functools import wraps
def coroutine(func):
    '''
    此装饰器是作者提供，不知道标准库中是否有与其功能相同的装饰器。
    该装饰器的功能就是提供预激协程的功能
    :param func:
    :return:
    '''
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen) # 预激协程！！！！
        return gen
    return primer

In [11]:
@coroutine
def averager2():
    total = 0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

a2 = averager2()
a2.send(1)

1.0

In [ ]:
a2.send(2)

1.5

很多框架都提供了处理协程的特殊装饰器，不过不是所有装饰器都用于预激协程，有些会提供其他服务，
例如勾入事件循环。比如说，异步网络库Tornado提供了tornado.gen装饰器。

**使用yield from句法（参见16.7节）调用协程时，会自动预激，因此与示例16-5中的@coroutine等装饰器不兼容。**
Python 3.4标准库里的asyncio.coroutine装饰器（第18章介绍）不会预激协程，因此能兼容yield from句法。

## 16.5 终止协程和异常处理
协程中未处理的异常会向上冒泡，传给next函数或send方法的调用方（即触发协程的对象）。

In [ ]:
a2.send('aaa')
# TypeError: unsupported operand type(s) for +=: 'int' and 'str'

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

### 终止协程方法一：使用哨符值，让协程退出
使用此方法的前提是，要在协程函数中处理异常！
```
my_coro.send(None)
my_coro.send(Ellipsis)
my_coro.send(StopIteration)
```

In [ ]:
my_av = averager2()
my_av.send(5.0)
my_av.send(10.0)

7.5

In [ ]:
getgeneratorstate(my_av)

'GEN_SUSPENDED'

In [ ]:
# 可以选用的哨符值： None  Ellipsis  StopIteration
my_av.send(None)
# 这里会被打断是因为我们没有在协程中处理异常！

TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'

In [ ]:
getgeneratorstate(my_av)
# 'GEN_CLOSED'

'GEN_CLOSED'

### 终止协程方法二：使用close()方法

In [ ]:
my_av2 = averager2()
my_av2.send(5.0)
my_av2.send(10.0)
getgeneratorstate(my_av2)

'GEN_SUSPENDED'

In [ ]:
my_av2.close()
getgeneratorstate(my_av2)

'GEN_CLOSED'

### generator.throw(exc_type[, exc_value[, traceback]])
致使生成器在暂停的yield表达式处抛出指定的异常。

如果生成器处理了抛出的异常，代码会向前执行到下一个yield表达式，
而产出的值会成为调用generator.throw方法得到的返回值。

如果生成器没有处理抛出的异常，异常会向上冒泡，传到调用方的上下文中。

### generator.close（）

致使生成器在暂停的yield表达式处抛出GeneratorExit异常。

如果生成器没有处理这个异常，或者抛出了StopIteration异常（通常是指运行到结尾），调用方不会报错。

如果收到GeneratorExit异常，生成器一定不能产出值，否则解释器会抛出RuntimeError异常。
生成器抛出的其他异常会向上冒泡，传给调用方。

---
### 如何使用throw和close

In [ ]:
# 自定义一个异常，为了演示方便
class DemoException(Exception):
    """
    为了演示定义的异常类型
    """

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received:{!r}'.format(x))
    raise RuntimeError('This line should never run')

In [ ]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [ ]:
exc_coro.send(11)

-> coroutine received:11


In [ ]:
# 如果传入的异常协程已经处理，就会继续运行
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [ ]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [ ]:
# 如果传入的异常协程没有处理，就会停止
exc_coro.throw(ZeroDivisionError)
# 而且异常会向上传递！

ZeroDivisionError: 

In [ ]:
getgeneratorstate(exc_coro)
# 'GEN_CLOSED'

'GEN_CLOSED'

In [ ]:
# 手动关闭！
exc_coro.close()

In [ ]:
getgeneratorstate(exc_coro)
# 'GEN_CLOSED'

'GEN_CLOSED'

In [ ]:
# 如果不管协程如何结束都想做些清理工作，要把协程定义体中相关的代码放入try/finally块中
def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received:{!r}'.format(x))
    finally:
        print('-> coroutine ending!')


In [ ]:
df = demo_finally()
next(df)
df.send(1)

-> coroutine started
-> coroutine received:1


In [ ]:
df.throw(ZeroDivisionError)

-> coroutine ending!


ZeroDivisionError: 

## 16.6 让协程返回值

In [20]:
# 测试之前的函数
av = averager2()
a = av.send(20)
b = av.send(40)
a,b

(20.0, 30.0)

In [21]:
# 定义一个求平均值的协程，让它返回一个结果
from collections import namedtuple
Result  = namedtuple('Result', 'count average')
@coroutine
def averager_result():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break #为了返回值，协程必须正常终止
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [22]:
coro_avg = averager_result()
coro_avg.send(10)
coro_avg.send(20)
coro_avg.send(30)
coro_avg.send(40)
coro_avg.send(None) # 终止协程
# 注意，return表达式的值会偷偷传给调用方，赋值给StopIteration异常的一个属性。
# 这样做有点不合常理，但是能保留生成器对象的常规行为——耗尽时抛出StopIteration异常

StopIteration: Result(count=4, average=25.0)

In [23]:
# 捕获StopIteration异常，获取averager返回的值
coro_avg = averager_result()
coro_avg.send(10)
coro_avg.send(20)
coro_avg.send(30)
coro_avg.send(40)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value

result

Result(count=4, average=25.0)

yield from结构会在内部自动捕获StopIteration异常。这种处理方式与for循环处理StopIteration异常的方式一样：循环机制使用用户易于理解的方式处理异常。对yield from结构来说，解释器不仅会捕获StopIteration异常，还会把value属性的值变成yield from表达式的值。可惜，我们无法在控制台中使用交互的方式测试这种行为，因为在函数外部使用yield from（以及yield）会导致句法出错。

## 16.7 使用yield from

它的作用比yield多很多，因此人们认为继续使用那个关键字多少会引起误解。在其他语言中，类似的结构使用*await*关键字，这个名称好多了，因为它传达了至关重要的一点：在生成器gen中使用yieldfrom subgen（　）时，subgen会获得控制权，把产出的值传给gen的调用方，即调用方可以直接控制subgen。与此同时，gen会阻塞，等待subgen终止。

In [26]:
# 回顾yield from的基本用法
def gen():
    yield from 'ABCDEFG'
    yield from 'HIJKLMN'

for g in gen():
    print(g)

A
B
C
D
E
F
G
H
I
J
K
L
M
N


In [29]:
# 回顾yield from的基本用法
def chain(*iteration):
    for it in iteration:
        yield from it

a = 'ABCDEFG'
b = 'HIJKLMN'
list(chain(a,b))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']

yield from的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，这样二者可以直接发送和产出值，还可以直接传入异常，而不用在位于中间的协程中添加大量处理异常的样板代码。有了这个结构，协程可以通过以前不可能的方式委托职责。

**把职责委托给子生成器的句法**

- 委派生成器：包含yield from <iterable>表达式的生成器函数。
- 子生成器：从yield from表达式中<iterable>部分获取的生成器。
- 调用方（客户端）：“调用方”这个术语指代调用委派生成器的客户端代码。

![coroutine](./img/委派生成器.png)

In [33]:
# BEGIN YIELD_FROM_AVERAGER
from collections import namedtuple

Result = namedtuple('Result', 'count average')

'''
yield from的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，这样二者可以直接发送和产出值，还可以直接传入异常，
而不用在位于中间的协程中添加大量处理异常的样板代码。有了这个结构，协程可以通过以前不可能的方式委托职责。
'''
# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield        # main函数中的客户代码发送的各个值绑定到这里的term变量上 
        if term is None:    # 至关重要的终止条件。如果不这么做，使用yield from调用这个协程的生成器会永远阻塞
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)  # 返回的Result会成为grouper函数中yield from表达式的值

# 委派生成器
def grouper(results, key):
    while True:  # 这个循环每次迭代时会新建一个averager实例；每个实例都是作为协程使用的生成器对象。
        results[key] = yield from averager()
        # grouper发送的每个值都会经由yield from处理，通过管道传给averager实例。grouper会在yield from表达式处暂停，
        # 等待averager实例处理客户端发来的值。averager实例运行完毕后，返回的值绑定到results[key]上。while循环会不断创建averager实例，处理更多的值。

# 客户端代码
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key) # group是调用grouper函数得到的生成器对象，传给grouper函数的第一个参数是results，用于收集结果；第二个参数是某个键。group作为协程使用。
        next(group) # 预激group协程
        for value in values:
            group.send(value) # 把各个value传给grouper。传入的值最终到达averager函数中term=yield那一行；grouper永远不知道传入的值是什么。
        group.send(None) # 重要！至关重要的终止条件。如果不这么做，使用yield from调用这个协程的生成器会永远阻塞。也就是获得不了返回值，results是空的。
        # 把None传入grouper，导致当前的averager实例终止，也让grouper继续运行，再创建一个averager实例，处理下一组值。
    #print(results)
    report(results)


def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print("{:2} {:5} averaging {:.2f} {}".format(
            result.count, group, result.average, unit
        ))


data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}


if __name__ == '__main__':
    main(data)

# END YIELD_FROM_AVERAGER

 9 boys  averaging 40.42 kg
 9 boys  averaging 1.39 m
10 girls averaging 42.04 kg
10 girls averaging 1.43 m


### 上面例子运行过程：

1. 外层for循环每次迭代会新建一个grouper实例，赋值给group变量；group是委派生成器。

2. 调用next(group)，预激委派生成器grouper，此时进入while True循环，调用子生成器averager后，在yield from表达式处暂停。

3. 内层for循环调用group.send(value)，直接把值传给子生成器averager。同时，当前的grouper实例（group）在yield from表达式处暂停。

4. 内层循环结束后，group实例依旧在yield from表达式处暂停，因此，grouper函数定义体中为results[key]赋值的语句还没有执行。

5. 如果外层for循环的末尾没有group.send(None)，那么averager子生成器永远不会终止，委派生成器group永远不会再次激活，因此永远不会为results[key]赋值。

6. 外层for循环重新迭代时会新建一个grouper实例，然后绑定到group变量上。前一个grouper实例（以及它创建的尚未终止的averager子生成器实例）被垃圾回收程序回收。

因为委派生成器相当于管道，所以可以把任意数量个委派生成器连接在一起：一个委派生成器使用yield from调用一个子生成器，而那个子生成器本身也是委派生成器，使用yield from调用另一个子生成器，以此类推。最终，这个链条要以一个只使用yield表达式的简单生成器结束；

任何yield from链条都必须由客户驱动，在最外层委派生成器上调用next(...)函数或.send(...)方法。可以隐式调用，例如使用for循环。

## 16.8 yield from的意义

把迭代器当作生成器使用，相当于把子生成器的定义体内联在yield from表达式中。此外，子生成器可以执行return语句，返回一个值，而返回的值会成为yieldfrom表达式的值。

### 六点说明了yield from的行为

1. 子生成器产出的值都直接传给委派生成器的调用方（即客户端代码）。

2. 使用send（）方法发给委派生成器的值都直接传给子生成器。如果发送的值是None，那么会调用子生成器的__next__（）方法。
如果发送的值不是None，那么会调用子生成器的send（）方法。
如果调用的方法抛出StopIteration异常，那么委派生成器恢复运行。任何其他异常都会向上冒泡，传给委派生成器。

3. 生成器退出时，生成器（或子生成器）中的return expr表达式会触发StopIteration(expr)异常抛出。

4. yield from表达式的值是子生成器终止时传给StopIteration异常的第一个参数。

5. 传入委派生成器的异常，除了GeneratorExit之外都传给子生成器的throw（）方法。如果调用throw（）方法时抛出StopIteration异常，委派生成器恢复运行。StopIteration之外的异常会向上冒泡，传给委派生成器。

6. 如果把GeneratorExit异常传入委派生成器，或者在委派生成器上调用close（　）方法，那么在子生成器上调用close（　）方法，如果它有的话。如果调用close（　）方法导致异常抛出，那么异常会向上冒泡，传给委派生成器；否则，委派生成器抛出GeneratorExit异常。

### 利用伪代码演示yield from做了什么

In [ ]:
# 利用伪代码演示yield from做了什么
RESULT = yield from EXPR  

# yield from 的简化伪代码 (不考虑其他类型的异常，以及.throw(...)和.close（　）方法)

# _i (迭代器) : 子生成器
# _y（产出的值）: 子生成器产出的值  
# _r（结果）: 最终的结果（即子生成器运行结束后yield from表达式的值）
# _s（发送的值）: 调用方发给委派生成器的值，这个值会转发给子生成器
# _e（异常）: 异常对象（在这段简化的伪代码中始终是StopIteration实例）

_i = iter(EXPR) # EXPR可以是任何可迭代的对象，因为获取迭代器_i（这是子生成器）使用的是iter（）函数。
try:
    _y = next(_i)  # 预激子生成器；结果保存在_y中，作为产出的第一个值。
except StopIteration as _e:
    _r = _e.value  # 如果抛出StopIteration异常，获取异常对象的value属性，赋值给_r——这是最简单情况下的返回值（RESULT）。
else:
    while 1:  # 运行这个循环时，委派生成器会阻塞，只作为调用方和子生成器之间的通道。
        _s = yield _y # 产出子生成器当前产出的元素；等待调用方发送_s中保存的值。注意，这个代码清单中只有这一个yield表达式。
        try:
            _y = _i.send(_s) # 尝试让子生成器向前执行，转发调用方发送的_s。
        except StopIteration as _e: # 如果子生成器抛出StopIteration异常，获取value属性的值，赋值给_r，然后退出循环，让委派生成器恢复运行。
            _r = _e.value
            break
RESULT = _r # 返回的结果（RESULT）是_r，即整个yield from表达式的值。
